In [1]:
# !mkdir -p /kaggle/working/image2biomassmodel/
# !cp /kaggle/input/image2biomassmodelv1/pytorch/default/1/csiro_multitask_ckpt_2025-12-18.pt /kaggle/working/image2biomassmodel/csiro_multitask_ckpt_2025-12-18.pt
# /kaggle/working/csiro_multitask_ckpt_2025-12-18.pt 

In [2]:
## The model

import torch
import torch.nn as nn 
import torchvision.models as models

class ResNetBackbone(nn.Module):
    def __init__(self, pretrained: bool = True):
        super().__init__()
        
        if pretrained:
            w = models.ResNet18_Weights.DEFAULT
        else:
            w = None
        
        # use resnet18 instead of resnet34
        m = models.resnet18(weights=w)

        # ResNet stem + layers
        self.conv1   = m.conv1
        self.bn1     = m.bn1
        self.relu    = m.relu
        self.maxpool = m.maxpool
        self.layer1  = m.layer1
        self.layer2  = m.layer2
        self.layer3  = m.layer3   # "higher" features
        self.layer4  = m.layer4   # final features
        self.avgpool = m.avgpool

        # use the **last** block in each layer so it works for 18/34/etc.
        self.out_dim_layer2 = m.layer2[-1].bn2.num_features
        self.out_dim_layer3 = m.layer3[-1].bn2.num_features
        self.out_dim_final  = m.fc.in_features  # 512 for resnet18

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x      = self.layer1(x)
        feat_l2 = self.layer2(x)
        feat_l3 = self.layer3(feat_l2)
        feat_l4 = self.layer4(feat_l3)

        # global pooling for final features
        f_final = self.avgpool(feat_l4)
        f_final = torch.flatten(f_final, 1)  # (B, 512)

        return feat_l2, feat_l3, f_final
    
class Image2BiomassModel(nn.Module):
    def __init__(self, num_species, pretrained=True):
        super().__init__()
        
        self.backbone = ResNetBackbone(pretrained=pretrained)
        
        out_l2 = self.backbone.out_dim_layer2
        out_l3 = self.backbone.out_dim_layer3
        out_final = self.backbone.out_dim_final
        
      
        def make_head_species(out_dim):
            return nn.Sequential(
            # nn.AdaptiveAvgPool2d(output_size=(1, 1)),   # pool layer3 to (B, 256, 1, 1)
            # nn.Flatten(1),                              # (B, 256)
                nn.Linear(in_features=out_final, out_features=256),
                nn.ReLU(inplace=True),
                nn.Dropout(0.4), #
                nn.Linear(256, 128),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2),
                nn.Linear(128, out_dim)
            )
            
        def make_head_ndvi(out_dim):
            return nn.Sequential(
                nn.Linear(out_final, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(0.3),
                nn.Linear(256, 128),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2),
                nn.Linear(128, out_dim)
            )
        
        def make_head_height(out_dim):
            return nn.Sequential(
                nn.Linear(out_final, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(0.3), # 
                nn.Linear(256, 128),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2), # 
                nn.Linear(128, out_dim)
            )
            
        def make_head_biomass(out_dim):
            return nn.Sequential(
                nn.Linear(out_final, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2), # 
                nn.Linear(256, 128),
                nn.ReLU(inplace=True),
                nn.Dropout(0.1), # ++
                nn.Linear(128, out_dim)
            )
        
        def make_head_clover_p(out_dim):
            # Clover presence (binary) -> output is a logit (no sigmoid here)
            return nn.Sequential(
                nn.Linear(out_final, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(0.3),
                nn.Linear(256, 128),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2),
                nn.Linear(128, out_dim)
            )

        def make_head_clover_mag(out_dim):
            # Clover magnitude (regression) -> predict same space as y (standardized log1p)
            return nn.Sequential(
                nn.Linear(out_final, 256),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2),
                nn.Linear(256, 128),
                nn.ReLU(inplace=True),
                nn.Dropout(0.1),
                nn.Linear(128, out_dim)
            )

        
        # custome heads for Image2BiomassModel
        self.head_species = make_head_species(num_species)      # Classfication
        self.head_ndvi    = make_head_ndvi(1)                   # Regression
        self.head_height  = make_head_height(1)                 # Regression
        self.head_biomass    = make_head_biomass(4)     # Dead, Green, Total, GDM
        self.head_clover_p   = make_head_clover_p(1)    # presence logit
        self.head_clover_mag = make_head_clover_mag(1)  # magnitude
       
        
    def forward(self, x):
        out_l2, out_l3, f_final = self.backbone(x)              # IN: torch.Size([1, 3, 256, 512]) -> OUT:
        
        out = {
            "species"   : self.head_species(f_final),                  # (B,C)
            "ndvi"      : self.head_ndvi(f_final).squeeze(-1),         # (B,)
            "height"    : self.head_height(f_final).squeeze(-1),       # (B,)
            "biomass"   : self.head_biomass(f_final),                  # (B,4)
            "clover_p_logit"    : self.head_clover_p(f_final).squeeze(-1),   # (B,)
            "clover_mag"        : self.head_clover_mag(f_final).squeeze(-1), # (B,)
        }
        
        return out

In [3]:
import torch

import cv2
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
from pathlib import Path

import pandas as pd

TARGET_H, TARGET_W = 256, 512

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

# optional torch versions (handy for unnormalize)
IMAGENET_MEAN_T = torch.tensor(IMAGENET_MEAN).view(3,1,1)
IMAGENET_STD_T  = torch.tensor(IMAGENET_STD).view(3,1,1)
        
class TestDataset(Dataset):
    def __init__(self, csv_path, images_root, tfms=None):
        self.df = pd.read_csv(csv_path).reset_index(drop=True)
        self.images_root = images_root
        
        # simple test transforms (no random aug)
        if tfms is None:
            self.img_tfms = T.Compose([
                T.ToTensor(),
                T.Resize((TARGET_H, TARGET_W), interpolation=InterpolationMode.BICUBIC),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ])
    
    def _transform_image(self, img_path: Path) -> torch.Tensor:
        img_bgr = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
        if img_bgr is None:
            raise FileNotFoundError(f"Image not found or unreadable: {img_path}")
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        return self.img_tfms(img_rgb)
      
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        
        # load image
        img_path = f"{self.images_root}/{row['image_path']}"
        img_t = self._transform_image(img_path)
        
        # which biomass component do we need for this row?
        target_name = row["target_name"]
        sample_id = row["sample_id"]
        
        return sample_id, target_name, img_t 

In [4]:
import torch
from torch.utils.data import DataLoader

@torch.inference_mode()
def _inv_transform(col: str, x: torch.Tensor, state: dict) -> torch.Tensor:
    """
    x: tensor on standardized (and maybe log1p) scale -> returns raw scale tensor
    """
    stats = state["stats"]
    cfg   = state.get("target_cfg", {})

    mu, sig = stats[col]
    x = x * sig + mu

    if cfg.get(col, {}).get("log1p", False):
        x = torch.expm1(x)
        x = torch.clamp(x, min=0.0)

    return x

# -- Dataloader --
test_ds = TestDataset(
    csv_path="/kaggle/input/csiro-biomass/test.csv",
    images_root="/kaggle/input/csiro-biomass"
)

test_loader = DataLoader(test_ds, batch_size=16, num_workers=2, pin_memory=True, shuffle=False)

# # -- Model -- 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ckpt_path = "/kaggle/working/image2biomassmodel/csiro_multitask_ckpt_2025-12-17.pt"
ckpt_path = "/kaggle/working/csiro_multitask_ckpt_2025-12-18.pt"
ckpt = torch.load(ckpt_path, map_location=device)
state = ckpt["state"]      # your preprocessing stats/config

preds_rows = []

# IMPORTANT: recreate the model architecture exactly like training
model = Image2BiomassModel(num_species=16, pretrained=False).to(device)
model.load_state_dict(ckpt["model_state"], strict=True)
model.to(device).eval()

# --- Inference -- 
with torch.inference_mode():
    for sids, target_nms, images in test_loader:
        images = images.to(device=device)

        outputs = model(images)

        # biomass head is now (B,4): [Dry_Dead_g, Dry_Green_g, Dry_Total_g, GDM_g]
        preds_other = outputs["biomass"]  # (B,4)

        # clover two-part
        clover_p   = torch.sigmoid(outputs["clover_p_logit"])  # (B,)
        clover_mag = outputs["clover_mag"]                     # (B,) z-space
        clover_raw_mag = _inv_transform("Dry_Clover_g", clover_mag, state)  # (B,)
        clover_raw = (clover_p * clover_raw_mag).clamp_min(0.0)            # (B,)

        # map ONLY for the 4-dim head
        name2idx_other = {
            "Dry_Dead_g":  0,
            "Dry_Green_g": 1,
            "Dry_Total_g": 2,
            "GDM_g":       3,
        }

        for i, (sid, tname) in enumerate(zip(sids, target_nms)):
            if tname == "Dry_Clover_g":
                value = float(clover_raw[i].item())
            else:
                col = name2idx_other[tname]
                raw_pred = _inv_transform(tname, preds_other[i, col], state)  # scalar tensor
                value = float(raw_pred.item())

            preds_rows.append({"sample_id": sid, "target": value})
  
# data_root = "/kaggle/input/csiro-biomass"
submit_csv_fl="submission.csv"
submission = pd.DataFrame(preds_rows)
submission.to_csv(f"{submit_csv_fl}", index=False)

In [5]:
# !cat submission.csv